<strong>1. Load mô hình và các thư viện cần dùng</strong>

In [9]:
from collections import Counter
from keras.models import load_model
import nltk
import numpy as np
import re

model = load_model("model_0.9593.h5")

NGRAM = 5
MAXLEN = 39

# Các ký tự có thể xuất hiện trong encoder
alphabet = ['\x00', ' '] + list('0123456789aàáảãạăằắẳẵặâầấẩẫậeèéẻẽẹêềếểễệiìíỉĩịoòóỏõọôồốổỗộơờớởỡợuùúủũụưừứửữựyỳýỷỹỵAÀÁẢÃẠĂẰẮẲẴẶÂẦẤẨẪẬEÈÉẺẼẸÊỀẾỂỄỆIÌÍỈĨỊOÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢUÙÚỦŨỤƯỪỨỬỮỰYỲÝỶỸỴ')

<strong>2. Hàm encoder và decoder tương ứng với mô hình đã huấn luyện</strong>

In [10]:
# Đệm '\x00' vào cuối của các cụm ngrams có độ dài < độ dài tối đa là 39
def _encoder_data(text):
    x = np.zeros((MAXLEN, len(alphabet)))
    for i, c in enumerate(text[:MAXLEN]):
        x[i, alphabet.index(c)] = 1
    if i < MAXLEN - 1 :
        for j in range(i + 1, MAXLEN):
            x[j, 0] = 1
    return x
def _decoder_data(x):
    x = x.argmax(axis = -1)
    return ''.join(alphabet[i] for i in x)

In [11]:
print(_encoder_data('Tôi tên là Nguyễn Thế Anh').shape)
print(_decoder_data(_encoder_data('Tôi tên là Nguyễn Thế Anh')))

ValueError: 'T' is not in list

<strong>3. Các hàm sẽ sử dụng để sửa lỗi chính tả trong văn bản Tiếng Việt</strong>

- Hàm tác câu thành các ngrams

In [12]:
def _nltk_ngrams(sentence, n, maxlen):
    list_ngrams = []
    list_words = sentence.split()
    num_words = len(list_words)

    if (num_words >= n):
        for ngram in nltk.ngrams(list_words, n):
            if len(' '.join(ngram)) <= maxlen:
                list_ngrams.append(ngram)
    else:
        list_ngrams.append(tuple(list_words))

    return list_ngrams

In [13]:
_nltk_ngrams('Xuwr ný ngoon ngữ tuw nhêin', NGRAM, MAXLEN)

[('Xuwr', 'ný', 'ngoon', 'ngữ', 'tuw'), ('ný', 'ngoon', 'ngữ', 'tuw', 'nhêin')]

- Hàm dự đoán ngram bằng mô hình

In [14]:
def _guess(ngram):
    text = ' '.join(ngram)
    preds = model.predict(np.array([_encoder_data(text)]))
    return _decoder_data(preds[0]).strip('\x00')

In [15]:
_guess(('Xuwr', 'ný', 'ngoon', 'ngữ', 'tuwj', 'nhêin'))

ValueError: 'X' is not in list